# A* Pathfinding v2: Grid Environment (Euclidean Cost)

This notebook demonstrates the use of the `GridEnvironment` for pathfinding on a regular grid (heightmap) loaded from processed data.

## Setup

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add project root to path
project_root = Path("../").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.terrain_navigate import AStar, GridEnvironment, EuclideanCost

## 1. Load Terrain Grid

In [ ]:
data_path = Path("data/processed_map.npy")
if not data_path.exists():
    raise FileNotFoundError("processed_map.npy not found. Please run preprocess_data.py first.")

Z = np.load(data_path)
print(f"Loaded map with shape: {Z.shape}")

plt.figure(figsize=(8, 6))
plt.imshow(Z, cmap='terrain', origin='lower')
plt.colorbar(label='Elevation (m)')
plt.title("Terrain Grid")
plt.show()

## 2. Initialize Grid Environment
The `GridEnvironment` efficiently handles grid-based neighbors.

In [ ]:
env = GridEnvironment(
    Z=Z,
    global_x_min=0.0,
    global_x_max=float(Z.shape[1]),
    global_y_min=0.0,
    global_y_max=float(Z.shape[0]),
    resolution=1.0
)

cost_fn = EuclideanCost()
astar = AStar()

## 3. Plan Path

In [ ]:
max_x, max_y = Z.shape[1], Z.shape[0]
start_raw = np.array([max_x * 0.1, max_y * 0.1, 0.0])
goal_raw = np.array([max_x * 0.9, max_y * 0.9, 0.0])

# Snap to nearest grid nodes
start_node = env.get_nearest_node(start_raw)
goal_node = env.get_nearest_node(goal_raw)

print(f"Start Node: {start_node}")
print(f"Goal Node: {goal_node}")

# Find path
path, cost = astar.find_path(start_node, goal_node, env, cost_fn, node_radius=1.5)

if path is None:
    print("No path found!")
else:
    print(f"Path found! Length: {len(path)} nodes, Cost: {cost:.2f}")

## 4. Visualize Path

In [ ]:
if path is not None:
    path_arr = np.array(path)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(Z, cmap='terrain', origin='lower')
    plt.colorbar(label='Elevation (m)')
    
    plt.plot(path_arr[:, 0], path_arr[:, 1], 'r-', linewidth=2, label='Path')
    plt.plot(start_node[0], start_node[1], 'go', markersize=10, label='Start')
    plt.plot(goal_node[0], goal_node[1], 'rx', markersize=10, label='Goal')
    
    plt.legend()
    plt.title("A* Path on Grid (Euclidean Cost)")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()